# SISTEMA DE RECOMENDACIÓN DE EJERCICIOS DE PROGRAMACIÓN

## 1. COMPRENCIÓN DEL PROBLEMA
Dentro de la Universidad Andrés Bello, los estudiantes de carreras de ingeniería deben cursar la asignatura de "Introducción a la Programación", ya que necesitan tener conocimientos básicos, medios o avanzados de programación, dependiendo de la carrera que estén cursando y en qué año van.

La problemática surge cuando ciertos estudiantes reprueban o sacan notas mínimas en las pruebas y trabajos de la asignatura. Siendo esta la primera asignatura de programación de ciertas carreras, puede marcar seriamente el desempeño del estudiante en el futuro, llegando a provocar que algunos tengan deficiencias en conocimientos esenciales para sus carreras.

Además, los estudiantes a menudo carecen de orientación personalizada para su aprendizaje, lo que puede resultar en un bajo rendimiento en los métodos de estudio, ocasionando un bajo conocimiento o baja nota en la asignatura.

Por tanto, este proyecto busca desarrollar e implementar un sistema de recomendación de ejercicios de programación para estudiantes universitarios de ingeniería de la Universidad Andrés Bello. Este sistema personalizará las sugerencias de ejercicios según el nivel de habilidad, preferencias individuales y objetivos de aprendizaje de cada estudiante.

## 2. COMPRENCIÓN DE LOS DATOS

In [ ]:
# MODULOS
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:

# FUNCIONES
def show_dataframe_dimension(dataframe: pd.DataFrame) -> None:
    # Muestra las dimensiones del dataframe, tanto numero de registros guardados y sus atributos.
    print(f'[+] Número de Filas: {dataframe.shape[0]}')
    print(f'[+] Número de Columnas: {dataframe.shape[1]}')
    print(f'[*] Nombre de las columnas: {list(dataframe.columns)}')


def show_column_data_types(dataframe: pd.DataFrame)-> None:
    # Muestra los tipos de datos que se guardan en cada columnas de atributos del dataframe.
    print(f"[+] Columnas númericas: {dataframe.select_dtypes(include=['int', 'float']).columns}\n")
    print(f"[+] Columnas categoricas: {dataframe.select_dtypes(include=['object']).columns}\n")


def show_missing_data(dataframe: pd.DataFrame) -> None:
    # Muestra las cantidades de datos faltantes o vacios persisten dentro del dataframe. 
    data_faltante = dataframe.isna().sum().where(lambda x: x > 0).dropna().reset_index()
    data_faltante.columns = ['columna', 'datos_faltantes']
    data_faltante['porcentaje'] = (data_faltante['datos_faltantes'] / dataframe.shape[0]) * 100
    print(data_faltante.sort_values(by='porcentaje'))


def rename_dataframe_column(nombre_columna: str) -> str:
    # Renombra los atributos en minusculas y elimina los espacios de un dataframe
    return nombre_columna.replace(' ', '_').replace('-', '_').lower()


In [ ]:
# RUTAS
DIR_DATA_ORIGINAL = './datos/data-original'
DIR_DATA_LIMPIA = './datos/data-limpia'

# El catalogo se conforma por aquellos ejercicios de programacion que estaban disponibles en la pagina web.
df_catalogo = pd.read_csv(f"{DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")

# La data_a son resgistros generales de los estudiantes durantes el semestre, es deicr, datos del curso en general.
df_data_a = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")

# La data_a_2021 son resgitros globales de los estudiantes al tratar de realizar los ejercicios de la pagina web.
df_data_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")

In [ ]:
### DIMENSIONES DE LOS DATASETS

# CATALOGO
print(f"Dataset 'Catalogo.csv' cuenta con: \n[+] {df_catalogo.shape[0]} Registros\n[+] {df_catalogo.shape[1]} Atributos")
print(f"Los atributos del 'Catalogo.csv' son: {list(df_catalogo.columns)}\n")

# DATA_A
print(f"Dataset 'Data_a.csv' cuenta con: \n[+] {df_data_a.shape[0]} Registros\n[+] {df_data_a.shape[1]} Atributos")
print(f"Los atributos del 'Data_a.csv' son: {list(df_data_a.columns)}\n")

# DATA_A_2021
print(f"Dataset 'Data_a_2021.csv' cuenta con: \n[+] {df_data_2021.shape[0]} Registros\n[+] {df_data_2021.shape[1]} Atributos")
print(f"Los atributos del 'Data_a_2021.csv' son: {list(df_data_2021.columns)}\n")

#### CATALOGO

| Campo         | Descripción                                                                                                                                               | Tipo  |   Valores     | 
|---------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|-------|---------------|
| hito          | Conjunto de ejercicios                                                                                                                                    | int   | [1,2,3,4]     |
| oid           | Identificador del ejercicio                                                                                                                               | str   | *             |
| nombre        | Nombre del ejercicio                                                                                                                                      | str   | *             |
| s1            | Nivel 4 en Algoritmos - Algoritmos Recursivos                                                                                                             | int   | binario [0,1] |
| s2            | Nivel 3 en Algoritmos - Algoritmos que combinan varias estructuras de datos                                                                               | int   | binario [0,1] |
| s3            | Nivel 2 en Algoritmos - Algoritmos que presentan información utilizando una estructura de datos                                                           | int   | binario [0,1] |
| s4            | Nivel 1 en Algoritmos - Algoritmos que no requieren una estructura de datos                                                                               | int   | binario [0,1] |
| k1            | Conocimientos en definir una clase con atributos y métodos.                                                                                               | int   | binario [0,1] |
| k2            | Conocimientos en agregar, quitar y buscar elementos dentro de una estructura de datos (cadenas o listas).                                                 | int   | binario [0,1] |
| k3            | Conocimientos en definir un bucle y determinar sus características en función del problema.                                                               | int   | binario [0,1] |
| k4            | Conocimientos en definir variables, usar operadores de comparación, construir expresiones condicionales, etc.                                             | int   | binario [0,1] |
| skill         | un escalar que indica la habilidad del estudiante (entero) => se define al transformar el binario componente de s1,s2,s3,s4 a entero                      | int   | (0, 12)       |
| knowledge     | un escalar que indica el conocimiento del estudiante (entero) => se define al transformar el binario componente de k1,k2,k3,k4 a entero                   | int   | (0, 12)       |
| complexity    | un escalar que indica la complejidad (entero) => en un número binario de 8 bits resultante de unir s1,s2,s3,s4,k1,k2,k3,k4 y transformarlo a entero       | int   | binario [0,1] |
| item          | Código alfabético asociado a la complejidad, combina habilidad (4 bits en ASCII) con conocimiento (4 bits en ASCII): 0->a,1->b,2->c, etc.                 | str   | *             |


#### DATA_A

| Campo         | Descripción                                                                                                                                               | Tipo  |   Valores     | 
|---------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|-------|---------------|
| rut           | Número de identificación ficticio del estudiante
| usuario       | anonimo
| correo        | anonimo
| nombre        | anonimo
| nrc           | nrc del curso en donde el estuiante vio la asignatura
| profesor      | anonimo
| i-(1, 6)      | 
| f-(1, 6)      |
| t-(1, 6)      |
| r-(1, 6)      |
| p-(1, 6)      |
| s-(1, 6)      |
| l-(1, 6)      |
| op-(1, 6)     |
| sv-(1, 6)     |
| lt            |
| tt            |
| pt            |
| game_score    |
| score         |
| score_a       |
| score_p       |
| score_d       |
| score_s       |
| tema2         |
| tema3         |
| tema4         |
| tema5         |
| tema6         |
| tema7         |
| tema8         |
| tema9         |
| hito-(1, 4)   |
| seq-(1, 4)    |
| c-(1, 4)      |
| e-(1, 4)      |
| clase         |
| exitosos      |
| fallidos      |
| envios        |
| videos        |
| programa      |
| sol-(1, 4)    |
| tarea1        |
| tarea2        |
| controles     |
| np            |
| examen        |
| final         |
| estado        |

#### DATA_A_2021

| Campo             | Descripción                                                                                      | Tipo   | Valores       |
|-------------------|--------------------------------------------------------------------------------------------------|--------|---------------|
| hito-(1, 4)       | Son los puntajes obtenidos al realizar los ejercicios referentes de ese hito                     | int    | [1,2,3,4]     |
| seq-(1, 4)        | Secuencia de Complejidad de Ejercicios (alfabética) del Hito-n                                   | str    | *             |
| c-(1, 4)          | Complejidad Numérica Total del Hito-n                                                            | str    | *             |
| nrc               | NRC del curso                                                                                    | int    | continuos     |
| exitosos          | Número de envíos exitosos                                                                        | int    | *             |
| fallidos          | Número de envíos fallidos                                                                        | int    | *             |
| envidos           | Número total de envíos                                                                           | int    | *             |
| e-(0, 52)         | Ejercicio que realizó la persona dentro de la plataforma                                         | int    | binario [0,1] |
| programa          | ID de la carrera que está cursando el estudiante en la UNAB                                      | str    | *             |
| sol1              | Nota del estudiante al finalizar el semestre                                                     | float  | [1.0, 7.0]    |
| id                | ID del registro enviado                                                                          | int    | continuos     |

In [ ]:
# VISUALIZACION: CATALOGO
df_catalogo.head()

In [ ]:
# VISUALIZACION: DATA_A
df_data_a.head()

In [ ]:
# VISUALIZACION: DATA_A_2021
df_data_2021.head()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: CATALOGO
df_catalogo.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A
df_data_a.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A_2021
df_data_2021.info()

In [ ]:
# DATOS FALTANTES: CATALOGO
data_faltante = df_catalogo.isna().sum().where(lambda x: x > 0).dropna().reset_index()
data_faltante.columns = ['columna', 'datos_faltantes']
data_faltante['porcentaje'] = (data_faltante['datos_faltantes'] / df_catalogo.shape[0]) * 100
data_faltante = data_faltante.sort_values(by='porcentaje')
data_faltante

# Observacion: El dataset de los ejercicios no contempla con datos vacios.Cuenta con 53 ejercicios.

In [ ]:
# DATOS FALTANTES: DATA_A
data_faltante = df_data_a.isna().sum().where(lambda x: x > 0).dropna().reset_index()
data_faltante.columns = ['columna', 'datos_faltantes']
data_faltante['porcentaje'] = (data_faltante['datos_faltantes'] / df_data_a.shape[0]) * 100
data_faltante = data_faltante.sort_values(by='porcentaje')
data_faltante

# Observacion: Para el dataset 'data_a.csv' tiene multiples columnas con datos faltantes, la mayoria tienen que ver con los ejercicios que realizaron los estudiantes. 
# Por tanto, se puede suponer que los estudiantes hacen mas ejercicios de los Hito 1 y 2, siendo que la mayoria de columnas del hito 3 y 4 tienen un 100% de datos faltantes. 

In [ ]:
# DATOS FALTANTES: DATA_A_2021
data_faltante = df_data_2021.isna().sum().where(lambda x: x > 0).dropna().reset_index()
data_faltante.columns = ['columna', 'datos_faltantes']
data_faltante['porcentaje'] = (data_faltante['datos_faltantes'] / df_data_2021.shape[0]) * 100
data_faltante = data_faltante.sort_values(by='porcentaje')
data_faltante

# Observacion: El dataset 'data_a_2021.csv' tienen el mismo problema que el anterior, se nota que existen semejanzas de atributos con mayores porcentajes de datos faltantes. 
# Las columnas que se repiten como Seq1, Seq2, Seq3 y Seq4, representan la secuencia de ejercicios realizados en cada hito por los estudiantes, es decir, 
# Al tener Seq3 y Seq4 todas sus datos vacios en esas columnas, se puede afirmar que los estudiantes solo realizaron ejercicos del Hito 1, en su mayoria, y algunos del Hito 2.

In [ ]:
# DATOS ESTADISTICOS: CATALOGO
df_catalogo.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A
df_data_a.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A_2021
df_data_2021.describe().T

In [ ]:
# CORRELACIÓN DE VARIABLES: CATALOGO
corr_matrix = df_catalogo.select_dtypes(include=['int', 'float']).corr()
corr_matrix["complexity"].sort_values(ascending=False)[:20]

# Nota: 
# Teniendo en cuenta que, si se tuviera que relacionar la complejidad del ejercicio que el estudiante realiza, su puntaje deberia estar influenciado por esta correlacion...
# Teniendo en cuenta, que las variables 'Skill' y 'Knowledge' vienen de las variables one-hot-encoded, tienen mucha correlacion, pero la complejidad esta muy influenciada por la habilidad o 'Skill' tenga el ejercicio.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A
corr_matrix = df_data_a.select_dtypes(include=['int', 'float']).corr()
corr_matrix["final"].sort_values(ascending=False)[:20]

# Observacion:
# Las variables 'final' y 'np' estan estrechamente correlacionados, esto se puede deber ya que se representan la nota final y nota de presentacion del estudiante al final del semestre.
# Tambien se puede observar como las solemnes 4 y 3 estan igualmente correlacionadas con la nota final del estudiante. Sin embargo, esto es ilogico, ya que si los estudiantes quisieran 
# mejorar su rendimiento academico y mejorar sus notas para eximir la asignatura, deberian realizar ejercicios de los hitos 3 y 4. Asi mismo, reducirian las probabilidades de reprobar.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A_2021
corr_matrix = df_data_2021.select_dtypes(include=['int', 'float']).corr()
corr_matrix["sol1"].sort_values(ascending=False)[:20]

# Nota:
# Al igual que el dataset anterior, solo se muestra ejercicios y la variable a correlacionar, en este caso, seria 'sol1' que representa el puntaje de la solemne 1 que saco el estudiante. 
# En este dataset, solo muestra los ejercicios que mas fluyo en la obtencion del puntaje de la 'sol1'. 

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: CATALOGO  
df_catalogo.hist(figsize=(22, 20))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A
df_data_a.hist(figsize=(28, 20))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A_2021
df_data_2021.hist(figsize=(22, 20))
plt.show()

### GRAFICOS: catalogo.csv

In [ ]:
sns.histplot(data=df_catalogo, x='knowledge')

### GRAFICOS: data_a.csv

In [ ]:
notas = df_data_a['estado'].value_counts()

def func(pct, allvalues):
    absolute = int(pct / 100.*np.sum(allvalues))
    return "{:.1f}%\n({:d} pers.)".format(pct, absolute)

fig, ax = plt.subplots()
pie = ax.pie(notas.values, labels=notas.index, autopct=lambda pct: func(pct, notas.values), labeldistance=1.1, colors=sns.color_palette('Set2'), explode=(0.1, 0.1), shadow=True)
ax.set_title(label="Resultados académicos de los estudiantes", fontdict={"fontsize":16}, pad=20)
ax.legend(pie[0], ["Aprobados", "Reprobados"], loc="upper right",  bbox_to_anchor=(1.1, 1))
plt.show()

In [ ]:
carreras = df_data_a['programa'].value_counts()
fig, ax = plt.subplots(figsize=(12, 5))  
sns.barplot(x=carreras.values, y=carreras.index, orient='h', hue=carreras.index, palette='Set2', dodge=False, ax=ax)

for container in ax.containers:
    ax.bar_label(container, fontsize=10)

plt.title('Total de estudiantes registrados por carrera', fontsize=16)
plt.xlabel('Carreras')
plt.ylabel('Número total de estudiantes participantes')
plt.show()

In [ ]:
programa_df = pd.DataFrame({'Aprobados': df_data_a[df_data_a['estado'] == 'A'].groupby('programa').size(), 'Reprobados': df_data_a[df_data_a['estado'] == 'R'].groupby('programa').size()})
programa_df.fillna(0.0, inplace=True)
programa_df = programa_df.astype(int)   # Aseguramos que todos los valores sean enteros
y = np.arange(len(programa_df.index))   # las ubicaciones de las etiquetas
height = 0.25                           # la altura de las barras
multiplier = 0

fig, ax = plt.subplots(figsize=(10, 6))

for attribute, measurement in programa_df.to_dict().items():
    offset = height * multiplier
    rects = ax.barh(y + offset, measurement.values(), height, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Añadir algunos textos para etiquetas, título y etiquetas personalizadas del eje x, etc.
ax.set_xlabel('Cantidad')
ax.set_title('Estudiantes aprobados y reprobados por programa')
ax.set_yticks(y + height / 2)
ax.set_yticklabels(programa_df.index)
ax.legend(loc='lower right')
ax.set_xlim(0, programa_df.max().max() + 10)  # Ajuste de límites del eje x

plt.show()

In [ ]:
sns.histplot(data=df_data_a, x='np')

### GRAFICOS: data_a_2021.csv

In [ ]:
sns.histplot(df_data_2021['sol1'])

In [ ]:
# Suponiendo que 'df_data_2021' es tu DataFrame
df_data_2021['sol1'] = df_data_2021['sol1'].astype(float)
df_data_2021['sol1'].unique()
# Suponiendo que df_data_2021 ya esté cargado y contenga los datos
rangos = [(1.0, 1.9), (2.0, 2.9), (3.0, 3.9), (4.0, 4.9), (5.0, 5.9), (6.0, 6.9), (7.0, 7.1)]

def contar_personas_en_rango(df, rango):
  rango_inferior, rango_superior = rango
  return df[(df['sol1'] >= rango_inferior) & (df['sol1'] < rango_superior)].shape[0]

conteo_por_rango = []
for rango in rangos:
  conteo = contar_personas_en_rango(df_data_2021, rango)
  conteo_por_rango.append((rango, conteo))

for rango, conteo in conteo_por_rango:
  rango_inferior, rango_superior = rango
  print(f"Rango: {rango_inferior:.1f} - {rango_superior:.1f}: {conteo} personas")

## 3. PREPARACIÓN DE LOS DATOS

In [1]:
# MODULOS
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# RUTAS
DIR_DATA_ORIGINAL = './datos/data-original'
DIR_DATA_LIMPIA = './datos/data-limpia'

warnings.filterwarnings('ignore')

#### EJERCICIOS

In [2]:
# CARGAR DATASETS DE CATALOGO DE EJERCICIOS
df_ejercicios = pd.read_csv(f"{DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")

# EJERCICOS HITOS EN ONE-HOT
# df_ejercicios = df_ejercicios.sort_values(by=["hito", "complexity"], ascending=True).reset_index()
df_ejercicios = pd.get_dummies(df_ejercicios, columns=['hito'], prefix='h', prefix_sep='', dtype=int)

# ORGANIZACION DE LAS VARIABLES BINARIAS EN BASE A SU NIVEL, ES DECIR; QUE S1, S2, S3, S4 EQUIVALEN A LOS NIVELES DE 1, 2, 3 Y 4 CORRESPONDIENTEMENTE.
# DE IGUAL MANERA, SE CAMBIO EL ORDEN DE LOS KNOWLEDGE QUE CAMBIA EL ORDEN DEJANDO 
df_ejercicios = df_ejercicios[['oid', 'nombre', 'h4', 'h3', 'h2', 'h1', 's1','s2','s3', 's4', 'k1', 'k2', 'k3', 'k4']]
df_ejercicios.columns = ['oid', 'nombre', 'h4', 'h3', 'h2', 'h1', 's4', 's3', 's2', 's1', 'k4', 'k3', 'k2', 'k1']

# # CONFIGURAR LA VARIABLE SCORE QUE CALCULA EL PUNTAJE DEL EJERCICIO EN BASE DE UN NUMERO BINAIRIO DE 12 BITS DE LOS DEMAS VARIBALES
# df_ejercicios['score'] = df_ejercicios.iloc[:, 3:15].apply(lambda x: int(''.join(map(str, x)), 2), axis=1)

# LA VARAIBLE IDX CORRESPONDE AL ORDEN ANTES DEL ORDENAMIENTO, ESTO ES PARA LA REFERENCIA DEL DATAFRAME DE REGISTROS DE LOS ALUMNOS (DATA_A_2021.CSV)
df_ejercicios.to_csv(f'{DIR_DATA_LIMPIA}/catalogo-de-ejercicios.csv', index=False)
df_ejercicios

,oid,nombre,h4,h3,h2,h1,s4,s3,s2,s1,k4,k3,k2,k1
0,0973dae0e1b74ab8baa8d94339ee3ae6,Cálculo del dígito verificador del rut,0,0,0,1,0,0,0,1,0,1,1,1
1,16f619db31204ded9418136c4587ddd8,Calculadora Geométrica,0,0,1,0,0,0,0,1,0,0,0,1
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,0,1,1,1
3,171b5e86d4fb47268f2692587fbec073,Números Primos,0,0,0,1,0,0,0,0,0,0,1,1
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1
5,24b87304a1eb4e95856da7f574a66fe8,Validador de Expresiones Matemáticas,1,0,0,0,1,0,0,0,0,1,1,1
6,29f15ef8dc32426f945f64e28c910a57,Suma de los divisores de un número,0,0,1,0,0,0,0,1,0,0,1,1
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,0,0,0,0,1
8,37e4f4a1e8174e9496d21b00d67fc8f1,Distancia Levenshtein,0,0,1,0,0,0,1,1,0,1,1,1
9,3d7d9e64ac0846caadc08dccf2538f55,Validar Secuencias de ADN,0,0,1,0,0,0,1,0,0,1,1,1


#### DATA_A & DATA_A_2021

In [3]:
df_data_a = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")
df_data_a = df_data_a.sort_values(by='rut', ascending=True)
df_data_a

,rut,usuario,correo,nombre,nrc,profesor,i1,f1,t1,r1,...,sol2,sol3,sol4,tarea1,tarea2,controles,np,examen,final,estado
283,0,*,*,*,17933,*,357.297418,421.598728,64.301310,1,...,6.8,5.1,6.0,3.5,7.0,5.8,5.86,5.9,5.9,A
393,1,*,*,*,17350,*,36.134093,133.304993,97.170900,2,...,6.9,7.0,7.0,7.0,6.4,3.4,6.47,6.5,6.5,A
99,2,*,*,*,17350,*,32.894832,118.491567,85.596734,1,...,6.0,4.5,5.4,5.5,1.0,4.4,4.20,6.0,4.7,A
59,3,*,*,*,17350,*,67.227459,125.003622,57.776163,1,...,4.7,3.7,4.1,3.0,1.0,3.7,3.12,6.6,4.2,A
284,4,*,*,*,17330,*,47.086279,79.606096,32.519816,1,...,6.6,4.7,5.3,6.5,6.2,6.7,5.84,6.0,5.9,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,462,*,*,*,17331,*,51.825712,106.851988,55.026277,1,...,6.1,6.2,6.3,3.5,5.4,7.0,5.72,5.7,5.7,A
374,463,*,*,*,17322,*,202.581834,671.856909,469.275075,3,...,5.8,5.8,5.7,6.9,6.7,7.0,6.26,6.3,6.3,A
437,464,*,*,*,17333,*,19.482461,73.030002,53.547541,3,...,5.7,6.8,6.4,7.0,7.0,7.0,6.71,6.7,6.7,A
239,465,*,*,*,17330,*,17.615900,74.230012,56.614112,2,...,4.8,4.8,4.0,7.0,6.9,7.0,5.67,5.3,5.6,A


In [4]:
df_data_a_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")
df_data_a_2021 = df_data_a_2021.sort_values(by='id', ascending=True).drop(labels=['id'], axis=1)
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0])) # 467 hasta (467+839)-1 
df_data_a_2021

,hito1,hito2,hito3,hito4,seq1,seq2,seq3,seq4,c1,c2,...,e46,e47,e48,e49,e50,e51,e52,programa,sol1,rut
0,17.0,0.0,0,0,abacabadadacbhbbadad,NaN,NaN,NaN,:1:2:1:3:3:2:23:17:3:3,NaN,...,0,0,0,0,0,0,0,UNAB22115,1.0,467
1,12.0,0.0,0,0,abacabadadacbh,NaN,NaN,NaN,:1:2:1:3:3:2:23,NaN,...,0,0,0,0,0,0,0,UNAB22115,1.0,468
2,11.0,0.0,0,0,abacadadacbh,NaN,NaN,NaN,:1:2:3:3:2:23,NaN,...,0,0,0,0,0,0,0,UNAB22115,1.0,469
3,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,UNAB22115,1.0,470
4,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,UNAB22510,1.0,471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,20.0,0.0,0,0,abacabadadacbhbbadadad,NaN,NaN,NaN,:1:2:1:3:3:2:23:17:3:3:3,NaN,...,0,0,0,0,0,0,0,UNAB12210,7.0,1301
835,14.0,0.0,0,0,abacabadadbhbbadad,NaN,NaN,NaN,:1:2:1:3:3:23:17:3:3,NaN,...,0,0,0,0,0,0,0,UNAB11500,7.0,1302
836,20.0,0.0,0,0,abacabadadacbhbbadadbd,NaN,NaN,NaN,:1:2:1:3:3:2:23:17:3:3:19,NaN,...,0,0,0,0,0,0,0,UNAB12100,7.0,1303
837,32.0,0.0,0,0,abacabadadacbhbbadadadabadbdadbd,NaN,NaN,NaN,:1:2:1:3:3:2:23:17:3:3:3:1:3:19:3:19,NaN,...,0,0,0,0,0,0,0,UNAB12210,7.0,1304


#### MATRICES DE FACTORIZACION

In [5]:
## MATRIZ DE EJERCICIOS (DATA_A)
matrix_factorizacion = pd.DataFrame(columns=['e' + str(i) for i in range(len(df_ejercicios))])
secuencia_ejercicios = df_data_a[['e1', 'e2', 'e3', 'e4']]


def crear_lista(n: int, lista_ejercicios: list) -> list:
    lista_resultado = [0] * n   # Crear una lista de n ceros
    if not lista_ejercicios:    # Si la lista está vacía, pasar a la siguiente lista
        return lista_resultado
    for ejercicio in lista_ejercicios:
        lista_resultado[ejercicio] += 1
    return lista_resultado


for index in range(len(secuencia_ejercicios)):
    ejercicios = secuencia_ejercicios.iloc[index]
    ejes1 = [int(elemento) for elemento in str(ejercicios.e1).split(':')[1:] if elemento.isdigit()] # Hito 1 
    ejes2 = [int(elemento) for elemento in str(ejercicios.e2).split(':')[1:] if elemento.isdigit()] # Hito 2 
    ejes3 = [int(elemento) for elemento in str(ejercicios.e3).split(':')[1:] if elemento.isdigit()] # Hito 3 
    ejes4 = [int(elemento) for elemento in str(ejercicios.e4).split(':')[1:] if elemento.isdigit()] # Hito 4 

    ejercicios_realizados = crear_lista(len(df_ejercicios), ejes1+ejes2+ejes3+ejes4)
    matrix_factorizacion.loc[len(matrix_factorizacion)] = ejercicios_realizados


matrix_factorizacion['rut'] = df_data_a['rut'].values
matrix_factorizacion = matrix_factorizacion[ ['rut'] + ['e' + str(i) for i in range(len(df_ejercicios))] ]

In [6]:
secuencia_ejercicios

,e1,e2,e3,e4
283,:25:17:4:26:29:23:0:22:10:3:42:18,:6:1:44,NaN,NaN
393,:25:17:4:26:29:23:0:22:10:3,NaN,NaN,NaN
99,:25:17:4:26:29:23:3:42:32:18:35,NaN,NaN,NaN
59,:25,NaN,NaN,NaN
284,:25:17:4:29:23:0:22:3:42:32:13,:1:44,NaN,NaN
...,...,...,...,...
259,:25:17:4:26:29:23:0:22:10:3:42:7:32:13:18:35,:6:1:44:11:37:28:12:21:38:8:9,NaN,NaN
374,:25:17:4:26:29:0:22:10:3:42:7:32:13:18:35,:6:1:44:37:38,NaN,NaN
437,:25:17:4:26:29:23:0:22:10:3:42:7:32:13:18:35,:6:1:44:11:37:28:12:21:38:8:9:2,NaN,NaN
239,:25:17:4:26:29:23:0:22:10:3:42:32:13,:1:44,NaN,NaN


In [7]:
## DATA_A_2021
df_data_a_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")
df_data_a_2021 = df_data_a_2021.drop(labels=['id'], axis=1)
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0]))

In [8]:
df_2021 = df_data_a_2021[['rut'] + ['e' + str(i) for i in range(len(df_ejercicios))]]
df_2021

,rut,e0,e1,e2,e3,e4,e5,e6,e7,e8,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,467,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,468,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,469,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,471,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1301,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
835,1302,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
836,1303,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
837,1304,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
matrix_factorizacion = pd.concat([matrix_factorizacion, df_2021], ignore_index=True)

In [10]:
matrix_factorizacion

,rut,e0,e1,e2,e3,e4,e5,e6,e7,e8,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,0,1,1,0,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1301,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1302,1302,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1303,1303,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1304,1304,1,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
matrix_factorizacion.to_csv(f'{DIR_DATA_LIMPIA}/matriz-de-factorizacion.csv', index=False)

#### REGISTROS DEL DATA_A

In [12]:
#### REGISTRO DEL CURSO
df_registro_curso = df_data_a[['rut','programa','nrc', 'sol1', 'sol2', 'sol3', 'sol4', 'tarea1', 'tarea2', 'controles', 'np', 'examen', 'final', 'estado']]
df_registro_curso.to_csv(f'{DIR_DATA_LIMPIA}/registro-curso.csv', index=False)
df_registro_curso

,rut,programa,nrc,sol1,sol2,sol3,sol4,tarea1,tarea2,controles,np,examen,final,estado
283,0,INGENIERIA INDUSTRIAL,17933,6.2,6.8,5.1,6.0,3.5,7.0,5.8,5.86,5.9,5.9,A
393,1,INGENIERIA CIVIL INDUSTRIAL,17350,7.0,6.9,7.0,7.0,7.0,6.4,3.4,6.47,6.5,6.5,A
99,2,INGENIERIA CIVIL INDUSTRIAL,17350,5.6,6.0,4.5,5.4,5.5,1.0,4.4,4.20,6.0,4.7,A
59,3,INGENIERIA CIVIL INDUSTRIAL,17350,3.9,4.7,3.7,4.1,3.0,1.0,3.7,3.12,6.6,4.2,A
284,4,INGENIERIA EN COMPUTACION E INFORMATICA,17330,4.6,6.6,4.7,5.3,6.5,6.2,6.7,5.84,6.0,5.9,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,462,INGENIERIA EN COMPUTACION E INFORMATICA,17331,6.6,6.1,6.2,6.3,3.5,5.4,7.0,5.72,5.7,5.7,A
374,463,INGENIERIA EN COMPUTACION E INFORMATICA,17322,5.5,5.8,5.8,5.7,6.9,6.7,7.0,6.26,6.3,6.3,A
437,464,INGENIERIA CIVIL INFORMATICA,17333,6.8,5.7,6.8,6.4,7.0,7.0,7.0,6.71,6.7,6.7,A
239,465,INGENIERIA EN COMPUTACION E INFORMATICA,17330,4.0,4.8,4.8,4.0,7.0,6.9,7.0,5.67,5.3,5.6,A


In [13]:
## PRUEBA DIAGNOSTICO
df_registro_prueba_diagnostico = df_data_a[[
    'rut',
    't1', 'r1', 'p1',
    't2', 'r2', 'p2',
    't3', 'r3', 'p3',
    't4', 'r4', 'p4',
    't5', 'r5', 'p5',
    't6', 'r6', 'p6',
    'op1', 'op2', 'op3', 'op4', 'op5', 'op6',
    'pt', 'game_score', 'score', 'score_a', 'score_p', 'score_d', 'score_s'
]]
df_registro_prueba_diagnostico.to_csv(f'{DIR_DATA_LIMPIA}/registro-prueba-diagnostico.csv', index=False)
df_registro_prueba_diagnostico

,rut,t1,r1,p1,t2,r2,p2,t3,r3,p3,...,op4,op5,op6,pt,game_score,score,score_a,score_p,score_d,score_s
283,0,64.301310,1,4,251.056409,3,10,224.745993,1,19,...,1,1,0,56,20.192022,8,5,4,2,3
393,1,97.170900,2,5,93.273640,2,9,108.991540,1,18,...,1,0,1,46,17.102511,8,4,4,2,3
99,2,85.596734,1,8,217.071180,7,16,150.406520,1,11,...,0,0,1,49,17.445343,8,5,4,3,2
59,3,57.776163,1,2,97.665937,1,4,187.063634,2,11,...,0,1,0,35,19.181718,6,3,3,1,2
284,4,32.519816,1,3,57.514397,2,5,75.568439,1,8,...,0,1,1,45,13.494646,10,6,4,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,462,55.026277,1,5,70.528988,1,16,118.709649,1,9,...,0,1,1,45,14.252496,10,7,4,3,3
374,463,469.275075,3,57,620.907184,7,124,262.061457,2,32,...,1,0,0,283,47.952695,5,4,2,1,2
437,464,53.547541,3,9,30.581918,1,4,335.801524,10,29,...,1,1,1,56,15.062700,6,3,3,2,3
239,465,56.614112,2,6,54.114042,2,12,226.322565,6,25,...,0,1,1,58,16.947992,5,3,2,2,0


In [14]:
## PLATAFORMA
df_registro_plataforma = df_data_a[[
    'rut', 
    'tema2', 'tema3', 'tema4', 'tema5', 'tema6', 'tema7', 'tema8',
    'hito1', 'hito2', 'hito3', 'hito4',
    'seq1', 'seq2', 'seq3', 'seq4',
    'c1', 'c2', 'c3', 'c4',
    'e1', 'e2', 'e3', 'e4',
    'exitosos', 'fallidos', 'envios', 'videos'
]]
df_registro_plataforma.to_csv(f'{DIR_DATA_LIMPIA}/registro-plataforma.csv', index=False)
df_registro_plataforma

,rut,tema2,tema3,tema4,tema5,tema6,tema7,tema8,hito1,hito2,...,c3,c4,e1,e2,e3,e4,exitosos,fallidos,envios,videos
283,0,1,0,0,0,0,0,0,18,3,...,NaN,NaN,:25:17:4:26:29:23:0:22:10:3:42:18,:6:1:44,NaN,NaN,15,24,39,0
393,1,0,0,0,0,0,0,0,14,0,...,NaN,NaN,:25:17:4:26:29:23:0:22:10:3,NaN,NaN,NaN,10,6,16,0
99,2,0,0,0,0,0,0,0,17,0,...,NaN,NaN,:25:17:4:26:29:23:3:42:32:18:35,NaN,NaN,NaN,11,20,31,0
59,3,0,0,0,0,0,0,0,1,0,...,NaN,NaN,:25,NaN,NaN,NaN,7,17,24,0
284,4,0,0,0,0,0,0,0,19,2,...,NaN,NaN,:25:17:4:29:23:0:22:3:42:32:13,:1:44,NaN,NaN,13,18,31,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,462,0,0,0,0,0,0,0,27,17,...,NaN,NaN,:25:17:4:26:29:23:0:22:10:3:42:7:32:13:18:35,:6:1:44:11:37:28:12:21:38:8:9,NaN,NaN,27,4,31,1
374,463,0,0,0,0,0,0,0,26,6,...,NaN,NaN,:25:17:4:26:29:0:22:10:3:42:7:32:13:18:35,:6:1:44:37:38,NaN,NaN,27,12,39,0
437,464,8,0,0,0,0,0,0,27,19,...,NaN,NaN,:25:17:4:26:29:23:0:22:10:3:42:7:32:13:18:35,:6:1:44:11:37:28:12:21:38:8:9:2,NaN,NaN,34,5,39,10
239,465,0,0,0,0,0,0,0,22,2,...,NaN,NaN,:25:17:4:26:29:23:0:22:10:3:42:32:13,:1:44,NaN,NaN,15,16,31,0


#### REGISTROS DEL DATA_A_2021

In [15]:
df_2021 = df_data_a_2021[['rut', 'programa', 'nrc', 'exitosos', 'fallidos', 'hito1', 'hito2', 'hito3', 'hito4', 'p1', 'p2', 'p3', 'p4', 'sol1']]
df_2021['hito1'] = df_2021['hito1'].astype(int)
df_2021['hito2'] = df_2021['hito2'].astype(int)
df_2021.to_csv(f'{DIR_DATA_LIMPIA}/registro-curso-2021.csv', index=False)
df_2021

,rut,programa,nrc,exitosos,fallidos,hito1,hito2,hito3,hito4,p1,p2,p3,p4,sol1
0,467,UNAB22115,1633,10,21,17,0,0,0,:25:17:4:26:29:23:0:22:10:3,NaN,NaN,NaN,1.0
1,468,UNAB22115,1632,7,9,12,0,0,0,:25:17:4:26:29:23:0,NaN,NaN,NaN,1.0
2,469,UNAB22115,1632,6,10,11,0,0,0,:25:17:26:29:23:0,NaN,NaN,NaN,1.0
3,470,UNAB22115,1633,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0
4,471,UNAB22510,10471,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1301,UNAB12210,10466,11,5,20,0,0,0,:25:17:4:26:29:23:0:22:10:3:42,NaN,NaN,NaN,7.0
835,1302,UNAB11500,10453,9,7,14,0,0,0,:25:17:4:26:29:0:22:10:18,NaN,NaN,NaN,7.0
836,1303,UNAB12100,10460,11,20,20,0,0,0,:25:17:4:26:29:23:0:22:10:3:35,NaN,NaN,NaN,7.0
837,1304,UNAB12210,10466,16,15,32,0,0,0,:25:17:4:26:29:23:0:22:10:3:42:7:32:13:18:35,NaN,NaN,NaN,7.0


## 4. MODELAMIENTO

## 5. EVALUCIÓN

## 6. IMPLEMENTACIÓN